# Demo's for AZ-2005

In this demo, we are going to use 2 plugins: one that retrieve feeds from the NY times rss and another that can write data to a file.

Semantic kernel creates a automtically a plan and invokes the correct functions.

At its simplest, function calling is merely a way for an AI to invoke a function with the right parameters.

Note: The Stepwise and Handlebars planners are still available in Semantic Kernel. However, we recommend using function calling for most tasks as it is more powerful and easier to use. Both the Stepwise and Handlebars planners will be deprecated in a future release of Semantic Kernel.

In [ ]:
#r "nuget:Microsoft.Extensions.Logging, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Debug, 8.0.0"

#r "nuget:Microsoft.SemanticKernel, 1.16.1"
#r "nuget:Microsoft.SemanticKernel.Plugins.Core, 1.16.2-alpha"

#r "nuget:dotenv.net, 3.0.0"
#r "nuget:SimpleFeedReader, 1.0.9"

In [ ]:
using dotenv.net;
using dotenv.net.Utilities;

DotEnv.Load();

var openai_endpoint = EnvReader.GetStringValue("OPENAI_ENDPOINT");
var openai_key = EnvReader.GetStringValue("OPENAI_KEY");

In [ ]:
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.Logging.Console;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;
using System.ComponentModel;
using SimpleFeedReader;

public class NewsPlugin {
    [KernelFunction("get_news")]
    [Description("Gets news items for today's date")]
    [return: Description("List of news stories")]
    public List<FeedItem> GetNews(Kernel kernel, string category) {
        var reader = new FeedReader();
        return  reader.RetrieveFeed($"https://rss.nytimes.com/services/xml/rss/nyt/{category}.xml").Take(5).ToList();
    }
}

public class ArchivePlugin {
    [KernelFunction("archive_data")]
    [Description("Saves data to a file on your computer")]

    public async Task WriteData(Kernel kernel, string filename, string data) 
    {
        await System.IO.File.WriteAllTextAsync($"{System.IO.Directory.GetCurrentDirectory()}\\{filename}", data);
    }
}


In [ ]:
var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
         "gpt-4o",         // Azure OpenAI Deployment Name. Make sure this is correct
         openai_endpoint,  // Azure OpenAI Endpoint
         openai_key);      // Azure OpenAI Key

builder.Services.AddLogging(c => c.AddDebug()
                                  .SetMinimumLevel(LogLevel.Trace) // don't do this in production!
                                  .AddConsole());

builder.Plugins.AddFromType<NewsPlugin>();
builder.Plugins.AddFromType<ArchivePlugin>();

var kernel = builder.Build();

var executionSettings = new OpenAIPromptExecutionSettings
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var prompt = "what's up today?";
var result = await kernel.InvokePromptAsync(prompt, new(executionSettings));
result.ToString().DisplayAs("text/markdown");


In [ ]:
var prompt = "What's happening in the world of sports?";
var result = await kernel.InvokePromptAsync(prompt, new(executionSettings));
result.ToString().DisplayAs("text/markdown");


In [ ]:
var prompt = "What's happening in the world of sports? Save this data to a file called sports-today.md";
var result = await kernel.InvokePromptAsync(prompt, new(executionSettings));
result.ToString().DisplayAs("text/markdown");